In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import mlp_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'mlp'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                300       
                                                                 
 dense_1 (Dense)             (None, 18)                234       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 553
Trainable params: 553
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 58s]
val_loss: 11.879406452178955

Best val_loss So Far: 8.637936115264893
Total elapsed time: 00h 19m 35s


# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 6, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 51457
========== Model 1 ==========
{'n_hidden_layers': 7, 'units_mult': 4, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1345
========== Model 2 ==========
{'n_hidden_layers': 1, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 2977
========== Model 3 ==========
{'n_hidden_layers': 3, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 26161
========== Model 4 ==========
{'n_hidden_layers': 1, 'units_mu

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME, n_epochs=N_EPOCHS,
                                           batch_size=BATCH_SIZE, n_top=TOP_N, es_min_delta=ES_MIN_DELTA,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 6, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 51457
Got score: 8.7346 (8.7197, 8.7495)
Got score: 8.6747 (8.4981, 8.8513)
Got score: 8.1989 (8.2496, 8.1481)
Got score: 8.6259 (8.6960, 8.5558)
Got score: 8.9995 (8.8819, 9.1171)


{'n_hidden_layers': 7, 'units_mult': 4, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 1345
Got score: 8.7630 (8.7670, 8.7591)
Got score: 8.4477 (8.4630, 8.4324)
Got score: 8.2663 (8.2639, 8.2687)
Got score: 8.5777 (8.6337, 8.5217)
Got score: 8.8704 (8.8973, 8.8436)


{'n_hidden_layers': 1, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 2977
Got score: 8.9142 (8.7611, 9.0673)
Got score: 8.4740 (8.4902, 8.4578)
Got score: 8.3222 (8.3521, 8.2924)
Got score: 8.5393 (8.5754, 8.5033)
Got score: 9.1286 (9.1831, 9.0740)


{'n_hidden_layers': 3, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 26161
Got score: 8.9348 (9.0001, 8.8696)
Got score: 8.6024 (8.6449, 8.5600)
Got score: 8.3109 (8.3472, 8.2746)
Got score: 8.5547 (8.6383, 8.4711)
Got score: 9.2707 (8.9977, 9.5437)


{'n_hidden_layers': 1, 'units_mult': 4, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 457
Got score: 8.7318 (8.6698, 8.7937)
Got score: 8.4401 (8.4360, 8.4443)
Got score: 8.2307 (8.2017, 8.2597)
Got score: 8.5592 (8.6372, 8.4811)
Got score: 9.0153 (8.9832, 9.0474)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.65,0.26,51457
1,8.59,0.22,1345
2,8.68,0.30,2977
3,8.73,0.33,26161
4,8.60,0.27,457
